In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erfc

def awgn(signal, snr_db):
    snr_linear = 10**(snr_db / 10)
    power_signal = np.mean(np.abs(signal)**2)
    noise_power = power_signal / snr_linear
    noise = np.sqrt(noise_power) * np.random.randn(*signal.shape)
    return signal + noise

def simulate_ask_m(m, snr_db_range, num_bits=100):
    """ Simule la modulation ASK-M et calcule le BER pour une gamme de SNR """
    bits_per_symbol = int(np.log2(m))
    num_symbols = num_bits // bits_per_symbol

    bits = np.random.randint(0, 2, num_bits)

    symbols = np.array([int("".join(map(str, bits[i:i+bits_per_symbol])), 2) for i in range(0, num_bits, bits_per_symbol)])

    modulated_signal = symbols * 2 - (m - 1)

    ber_results = []

    for snr_db in snr_db_range:
        received_signal = awgn(modulated_signal, snr_db)

        detected_symbols = np.round((received_signal + (m - 1)) / 2).astype(int)
        detected_symbols = np.clip(detected_symbols, 0, m - 1)

        detected_bits = np.array([list(f"{sym:0{bits_per_symbol}b}") for sym in detected_symbols], dtype=int).flatten()

        ber = np.mean(detected_bits != bits)
        ber_results.append(ber)

    return ber_results

snr_db_range = np.arange(0, 30, 2)

ber_ask2 = simulate_ask_m(2, snr_db_range)
ber_ask16 = simulate_ask_m(16, snr_db_range)
ber_ask64 = simulate_ask_m(64, snr_db_range)

plt.figure(figsize=(8, 6))
plt.semilogy(snr_db_range, ber_ask2, 'o-', label='ASK-2')
plt.semilogy(snr_db_range, ber_ask16, 's-', label='ASK-16')
plt.semilogy(snr_db_range, ber_ask64, 'd-', label='ASK-64')
plt.xlabel("SNR (dB)")
plt.ylabel("Taux d'Erreur Binaire (BER)")
plt.title("Courbes BER en fonction du SNR pour ASK-2, ASK-16 et ASK-64")
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()
